In [1]:
from astropy.io import fits
import pandas as pd
from astropy.coordinates import SkyCoord
import astropy.units as u
import matplotlib.pyplot as plt
import numpy as np
from scipy.cluster.vq import vq, kmeans
import scipy
from astroquery.gaia import Gaia
import os

import match

# Global Parameters
BAND = 'i'
PSF_DATA_FILEPATH = "../../psf_data/psf_y3a1-v29.fits"
RESULTS_FILEPATH = "../results/"
TOTAL_SUBSAMPLE_SIZE = 10000
MATCH_LIM = 1 * u.arcsec
INT_DATA_PATH = "../../int_data/"

In [2]:
def read_des_fits(file_path, band, n = int(1e6)):
    """
    Read in the DES fits file and return a pandas dataframe with ra, dec, mag and band columns.
    Args: 
        file_path: path to the fits file
        band: band to be read in
        n: subsample of stars to be read in within specified band
    Returns:
        des: pandas dataframe with ra, dec, mag and band columns
    """
    
    # Read in the fits file and close it
    hdul = fits.open(file_path)
    
    # hdul[1].data is a numpy recarray. Get the ra, dec, mag and band columns   
    cols = ['ra', 'dec', 'mag', 'band']
    zidx = np.random.choice(np.where(hdul[1].data['band'] == band)[0], size = n, replace = False)
    data = {col: hdul[1].data[col][zidx] for col in cols}
    hdul.close()

    des = pd.DataFrame(data)
    

    # Combine ra and dec into a sky coord array
    des['coord'] = SkyCoord(ra=des['ra'], dec=des['dec'], unit = 'deg')
    return des

print("Starting DES Gaia Crossmatch for Band " + str(BAND) + ".")

# Read in DES Data
des = read_des_fits(PSF_DATA_FILEPATH, BAND, n = TOTAL_SUBSAMPLE_SIZE)
print("Data read in.")

Starting DES Gaia Crossmatch for Band i.
Data read in.


In [3]:
# Load centroids array from int_data
centroids = np.load(INT_DATA_PATH + "centroids.npy")

In [4]:
cluster_num_array, cluster_info = match.get_assignments(des, centroids)

Stars clustered.


In [6]:
for cluster_num in range(3):
    clust0_info = cluster_info.loc[cluster_num]
    print("R = {:.3f}".format(clust0_info["max_dist"]), end = ' | ')
    gaia0_tab = match.query_gaia_for_cluster(clust0_info["centroids"][0], 
                                       clust0_info["centroids"][1], 
                                       clust0_info["max_dist"],
                                       verbose=True)
    print("Queried.", end = ' ')
    
    # Save the gaia table in int_data
    gaia0_tab.to_feather(INT_DATA_PATH + "gaia/" + "gaia" + str(cluster_num) + ".feather")
    print("Saved.")

R = 2.971 | 
    select top 1000000 ra, dec, phot_g_mean_mag, in_qso_candidates, in_galaxy_candidates, non_single_star, astrometric_excess_noise
    FROM gaiadr3.gaia_source
    WHERE 1 = CONTAINS(
    POINT(40.61000072213061, -59.887512128635855),
    CIRCLE(ra, dec, 2.970643848112104))
    order by source_id
    
Queried. Saved.
R = 3.253 | 
    select top 1000000 ra, dec, phot_g_mean_mag, in_qso_candidates, in_galaxy_candidates, non_single_star, astrometric_excess_noise
    FROM gaiadr3.gaia_source
    WHERE 1 = CONTAINS(
    POINT(321.63726674557637, -43.23227424475044),
    CIRCLE(ra, dec, 3.253305118428657))
    order by source_id
    


/home/shrihan/.conda/envs/weaklens2/lib/python3.10/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


Queried. Saved.
R = 2.167 | 
    select top 1000000 ra, dec, phot_g_mean_mag, in_qso_candidates, in_galaxy_candidates, non_single_star, astrometric_excess_noise
    FROM gaiadr3.gaia_source
    WHERE 1 = CONTAINS(
    POINT(81.8946918115738, -25.22550347558047),
    CIRCLE(ra, dec, 2.1674281096344767))
    order by source_id
    


/home/shrihan/.conda/envs/weaklens2/lib/python3.10/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


Queried. Saved.


/home/shrihan/.conda/envs/weaklens2/lib/python3.10/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


In [3]:
import pandas as pd
INT_DATA_PATH = "../../int_data/"
pd.read_feather(INT_DATA_PATH + "gaia/" + "gaia" + str(0) + ".feather")

,ra,dec,in_qso_candidates,in_galaxy_candidates,non_single_star,is_star
0,40.385375,-63.935587,False,False,0,True
1,40.379013,-63.933067,False,False,0,True
2,40.542765,-63.930190,False,False,0,True
3,40.537682,-63.934508,False,False,0,True
4,40.654792,-63.941125,False,False,0,True
...,...,...,...,...,...,...
234234,40.126497,-55.847247,False,False,0,True
234235,40.110284,-55.842860,True,False,0,True
234236,41.110041,-55.843370,False,False,0,True
234237,41.096716,-55.852361,False,False,0,True
